In [ ]:
## Following along with https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
## Using "Multivariate Multi-Step LSTM Models" Section

## Helpful links
## https://stackoverflow.com/questions/54354467/how-to-shape-large-dataframe-for-pythons-keras-lstm

In [36]:
import pandas as pd
import numpy as np
from pathlib import Path

from numpy import array
from numpy import hstack 

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
## Load a dataset 
symbol = 'GME'
import_path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
import_data = load_obj(import_path)
dataframe = import_data['GME']
#import_data

In [19]:
## Testing dataframe , last 100 rows
df = dataframe.tail(100).copy()
df.head()

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-11,158.43,159.05,154.62,158.78,158.78,948632.0,948632.0,0.35,0.221,157.48333,...,163764.0,4129.0,245603.0,35905.0,0.0,107590.0,0.595573,0.935142,0.216667,0.178905
2021-08-12,159.88,164.25,157.38,162.35,162.35,1330235.0,1330235.0,2.47,1.545,161.32667,...,229882.0,1846.0,379840.0,51356.0,0.0,142536.0,0.515761,0.945731,0.266667,0.237650
2021-08-13,160.47,163.55,157.57,162.52,162.52,1018170.0,1018170.0,2.05,1.277,161.21333,...,167441.0,1444.0,263031.0,31970.0,0.0,72643.0,0.551979,0.952349,0.700000,0.207610
2021-08-16,161.00,169.88,159.16,163.93,163.93,1560239.0,1560239.0,2.93,1.820,164.32333,...,297405.0,128.0,477601.0,63270.0,0.0,135990.0,0.611670,0.958968,0.933333,0.305073
2021-08-17,161.19,169.47,161.10,163.55,163.55,1216017.0,1216017.0,2.36,1.464,164.70667,...,204328.0,4408.0,325228.0,44260.0,0.0,118590.0,0.857143,0.961615,0.883333,0.453271


In [20]:
## Need to reorder columns - close price must be final (end column) before data re-shaping 

## Copy close column
df_close = df[['close','adjClose']]
df_close.head()

,close,adjClose
Date,,
2021-08-11,158.78,158.78
2021-08-12,162.35,162.35
2021-08-13,162.52,162.52
2021-08-16,163.93,163.93
2021-08-17,163.55,163.55


In [21]:
## Drop 'close' and 'adjClose' from df 
df = df.drop(columns=['close','adjClose'])
df.head()

,open,high,low,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,
2021-08-11,158.43,159.05,154.62,948632.0,948632.0,0.35,0.221,157.48333,0.00221,6624.0,163764.0,4129.0,245603.0,35905.0,0.0,107590.0,0.595573,0.935142,0.216667,0.178905
2021-08-12,159.88,164.25,157.38,1330235.0,1330235.0,2.47,1.545,161.32667,0.01545,125.0,229882.0,1846.0,379840.0,51356.0,0.0,142536.0,0.515761,0.945731,0.266667,0.237650
2021-08-13,160.47,163.55,157.57,1018170.0,1018170.0,2.05,1.277,161.21333,0.01277,1227.0,167441.0,1444.0,263031.0,31970.0,0.0,72643.0,0.551979,0.952349,0.700000,0.207610
2021-08-16,161.00,169.88,159.16,1560239.0,1560239.0,2.93,1.820,164.32333,0.01820,22314.0,297405.0,128.0,477601.0,63270.0,0.0,135990.0,0.611670,0.958968,0.933333,0.305073
2021-08-17,161.19,169.47,161.10,1216017.0,1216017.0,2.36,1.464,164.70667,0.01464,2020.0,204328.0,4408.0,325228.0,44260.0,0.0,118590.0,0.857143,0.961615,0.883333,0.453271


In [22]:
## Add close column back
df['close'] = df_close['close']
df.head()

,open,high,low,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,...,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal,close
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-11,158.43,159.05,154.62,948632.0,948632.0,0.35,0.221,157.48333,0.00221,6624.0,...,4129.0,245603.0,35905.0,0.0,107590.0,0.595573,0.935142,0.216667,0.178905,158.78
2021-08-12,159.88,164.25,157.38,1330235.0,1330235.0,2.47,1.545,161.32667,0.01545,125.0,...,1846.0,379840.0,51356.0,0.0,142536.0,0.515761,0.945731,0.266667,0.237650,162.35
2021-08-13,160.47,163.55,157.57,1018170.0,1018170.0,2.05,1.277,161.21333,0.01277,1227.0,...,1444.0,263031.0,31970.0,0.0,72643.0,0.551979,0.952349,0.700000,0.207610,162.52
2021-08-16,161.00,169.88,159.16,1560239.0,1560239.0,2.93,1.820,164.32333,0.01820,22314.0,...,128.0,477601.0,63270.0,0.0,135990.0,0.611670,0.958968,0.933333,0.305073,163.93
2021-08-17,161.19,169.47,161.10,1216017.0,1216017.0,2.36,1.464,164.70667,0.01464,2020.0,...,4408.0,325228.0,44260.0,0.0,118590.0,0.857143,0.961615,0.883333,0.453271,163.55


In [24]:
## Close is now at the end of the dataframe 
## Create function
def reorder_columns(dataframe):
    df = dataframe.copy()
    df_close = df[['close','adjClose']]
    df = df.drop(columns=['close','adjClose'])
    df['close'] = df_close['close']
    return df

In [25]:
symbol = 'GME'

import_path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
import_data = load_obj(import_path)
dataframe = import_data[symbol]

dataframe = reorder_columns(dataframe)
dataframe.head()

,open,high,low,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,...,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal,close
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-02,26.629999,26.709999,25.590000,1840600.0,1840600.0,-0.71,-2.666,26.07333,-0.02666,1714.0,...,152.0,277309.0,11326.0,0.0,34626.0,0.692824,0.607545,-0.933333,0.180908,25.920000
2016-02-03,26.110001,26.280001,24.750000,2223400.0,2223400.0,0.06,0.230,25.73333,0.00230,3045.0,...,14.0,523222.0,9575.0,0.0,26571.0,0.714956,0.657842,-0.933333,0.251669,26.170000
2016-02-04,26.059999,27.190001,25.379999,2000200.0,2000200.0,0.95,3.645,26.52667,0.03645,0.0,...,0.0,325811.0,17331.0,0.0,35015.0,0.591549,0.767042,-0.950000,0.666889,27.010000
2016-02-05,27.010000,27.180000,26.430000,1545800.0,1545800.0,-0.18,-0.666,26.81333,-0.00666,133.0,...,0.0,215572.0,6611.0,0.0,14960.0,0.617706,0.796823,-0.950000,0.532710,26.830000
2016-02-08,26.549999,27.959999,26.500000,2944800.0,2944800.0,1.34,5.047,27.45000,0.05047,4865.0,...,30.0,496626.0,27972.0,0.0,66574.0,0.491616,0.857710,-0.933333,0.795060,27.889999


In [26]:
## Next, scale the DF. Then, shape the DF for use with keras models 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [34]:
test_df = dataframe.copy()
#scaled_df = scaler.fit_transform(test_df)
scaled_df = test_df

In [37]:
## Attempt shaping dataset 
sequences = 5
result = [] 

for i in range(0,len(scaled_df)):
    # every group must have the same length, so if current loop position i + number 
    # of sequences is higher than df length, breaks
    if i+sequences <= len(scaled_df):
        # this will add into the list as [[R1a,R1b...R1t],[R2a,R2b...R2t],...[R5a,R5b...R5t]]
        result.append(scaled_df[i:i+sequences].values)
# Converting to array + keras takes float32 better than 64
train_x = np.array(result).astype('float32')
# making the y into same length as X
train_y = np.array(y.tail(train_x.shape[0]).values)

train_x.shape, train_y.shape

NameError: name 'y' is not defined

In [38]:
## Not working - try seperating x and y features 

In [39]:
symbol = 'GME'

import_path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
import_data = load_obj(import_path)
dataframe = import_data[symbol]

In [40]:
df = dataframe.copy()
df_close = df[['close']]
df = df.drop(columns=['close','adjClose'])
#df['close'] = df_close['close']
df_close.head()

,close
Date,
2016-02-02,25.920000
2016-02-03,26.170000
2016-02-04,27.010000
2016-02-05,26.830000
2016-02-08,27.889999


In [41]:
df.head()

,open,high,low,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,
2016-02-02,26.629999,26.709999,25.590000,1840600.0,1840600.0,-0.71,-2.666,26.07333,-0.02666,1714.0,117857.0,152.0,277309.0,11326.0,0.0,34626.0,0.692824,0.607545,-0.933333,0.180908
2016-02-03,26.110001,26.280001,24.750000,2223400.0,2223400.0,0.06,0.230,25.73333,0.00230,3045.0,212359.0,14.0,523222.0,9575.0,0.0,26571.0,0.714956,0.657842,-0.933333,0.251669
2016-02-04,26.059999,27.190001,25.379999,2000200.0,2000200.0,0.95,3.645,26.52667,0.03645,0.0,156570.0,0.0,325811.0,17331.0,0.0,35015.0,0.591549,0.767042,-0.950000,0.666889
2016-02-05,27.010000,27.180000,26.430000,1545800.0,1545800.0,-0.18,-0.666,26.81333,-0.00666,133.0,115038.0,0.0,215572.0,6611.0,0.0,14960.0,0.617706,0.796823,-0.950000,0.532710
2016-02-08,26.549999,27.959999,26.500000,2944800.0,2944800.0,1.34,5.047,27.45000,0.05047,4865.0,257398.0,30.0,496626.0,27972.0,0.0,66574.0,0.491616,0.857710,-0.933333,0.795060


In [44]:
X = df
y = df_close

In [45]:
## Split data
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [46]:
## Fit scaler
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [47]:
# ## Attempt shaping dataset 
# sequences = 5
# result = [] 

# for i in range(0,len(scaled_df)):
#     # every group must have the same length, so if current loop position i + number 
#     # of sequences is higher than df length, breaks
#     if i+sequences <= len(scaled_df):
#         # this will add into the list as [[R1a,R1b...R1t],[R2a,R2b...R2t],...[R5a,R5b...R5t]]
#         result.append(scaled_df[i:i+sequences].values)
# # Converting to array + keras takes float32 better than 64
# train_x = np.array(result).astype('float32')
# # making the y into same length as X
# train_y = np.array(y.tail(train_x.shape[0]).values)

# train_x.shape, train_y.shape

In [ ]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array